In [24]:

import os
import io
import json
import requests
import pandas as pd
import numpy as np
import time as time

Prepare Log File to which TV shows and other popular media will be registered.

In [436]:
my_dir = "entertainment/"
bin_dir = "bin/"
path_to_log_file = my_dir + bin_dir + "logs.csv"

dfhead = ['Category', 'Title', 'Date', 'Meta', 'url', 'score', 'comments']

In [441]:
if not os.path.exists(path_to_log_file):
    if not os.path.exists(my_dir):
        os.makedirs(my_dir + bin_dir)
    with open(path_to_log_file, mode='w') as log_file:
        log_file.write(pd.DataFrame(columns=dfhead).to_csv())
        log_file.close()

Import Flask to build an interactive UI

In [550]:
from flask import Flask, render_template

In [437]:
def date_now(delimeter='-'):
    time_struct = time.gmtime()
    date_str = str(time_struct.tm_mday) + f"{delimeter}" + str(time_struct.tm_mon) + f"{delimeter}" + str(time_struct.tm_year)
    return date_str

def get_all_titles(data):
    posts = []
    for post in data['data']['children']:
        x = post['data']['title']
        posts.append(x)
    return posts

def get_more_info(category=None, data=None):
    #EXTRACTING THE TITLE, URL, SCORE AND NUMBER OF COMMENTS
    meta_info = {}
    for post in data['data']['children']:
        meta_info[post['data']['title']] = {'Category': category, 'Title': post['data']['title'], 'Date': date_now(), 'Meta': 'None', 
        'url':post['data']['url'],'score':post['data']['score'],'comments':post['data']['num_comments']}
    df = pd.DataFrame.from_dict(meta_info, orient='index')
    return df    

In [331]:
headers = {
    'User-agent': 'benkimz'
}

Request a list of 15 items per genre

In [439]:
LIMIT = '25'
year = '2022'

In [535]:
all_music = requests.get(url = f'https://www.reddit.com/r/Music/top.json?limit={LIMIT}&t={year}', headers=headers).json()
new_music = requests.get(url = f'https://www.reddit.com/r/NewMusic/top.json?limit={LIMIT}&t={year}', headers=headers).json()
tv_shows = requests.get(url = f'https://www.reddit.com/r/television/top.json?limit={LIMIT}&t={year}', headers=headers).json()
new_tv_shows = requests.get(url = f'https://www.reddit.com/r/televisionsuggestions/top.json?limit={LIMIT}&t={year}', headers=headers).json()

Check items in the Log Files to update register

In [545]:
def add_to_logs(category, data, limit=1000):
    rich_data = get_more_info(category=category, data=data).copy()
    data_frame = pd.read_csv(path_to_log_file).filter(items=dfhead)
    try:
        if limit > (int(len(data)) - 1):
            limit = int(len(data)) - 1
        for i in range(0, limit):
            data_frame.loc[data_frame.shape[0]+1] = list(np.array(rich_data)[i])
        with open(path_to_log_file, mode='w', encoding='utf-8') as data_log:
            data_log.write(data_frame.to_csv())
            data_log.close()
    except Exception as E:
        return False

data = {
    "A": {
        "category": "Music", 
        "data": all_music
    }, 
    "B": {
        "category": "Music", 
        "data": new_music
    }, 
    "C": {
        "category": "TV shows", 
        "data": tv_shows
    }, 
    "D": {
        "category": "TV shows", 
        "data": new_tv_shows
    }
}

MIN_SCORE = 450
MIN_COMMENTS = 450
populars = set()

dframe = pd.read_csv(path_to_log_file).filter(items=dfhead)
if len(dframe) == 0:
    for item in data:
        add_to_logs(data[item]['category'], data[item]['data'])
else:
    response_data = [(title, data[d]['data']['data']['children'][0]['data']['url'], data[d]['data']['data']['children'][0]['data']['score'], data[d]['data']['data']['children'][0]['data']['num_comments']) for title in get_all_titles(data[d]['data']) for d in data]
    response_obj = [{title: {'category': data[d]['category'], "score": score, "comments": comments, "info": data[d]['data']}} for title, _, score, comments in response_data]
    saved_titles = list(dframe['Title'])
    for meta in response_obj:
        for meta_data in meta:
            if meta[meta_data]['score'] > MIN_SCORE or meta[meta_data]['score'] == MIN_SCORE:
                if meta[meta_data]['comments'] > MIN_COMMENTS or meta[meta_data]['comments'] == MIN_COMMENTS:
                    if not meta_data in populars and not meta_data in saved_titles:
                        populars.add(meta_data)
                        add_to_logs(meta[meta_data]['category'], meta[meta_data]['info'])
                else:
                    pass
            else:
                pass

View a list of some of the latest TV shows and music whose popularity is growing fast

In [619]:
populars = dframe[:75]
print(f'Available information include {populars.keys()}')
print(f'Selected items are {dframe.shape[0]} in total')
print(f'Creteria for popularity takes into account the number of comments and an overall score')
print('Some of the trending items include:')
print(dframe[:50])

Available information include Index(['Category', 'Title', 'Date', 'Meta', 'url', 'score', 'comments'], dtype='object')
Selected items are 55 in total
Creteria for popularity takes into account the number of comments and an overall score
Some of the trending items include:
    Category                                              Title       Date  \
0      Music  Pink Floyd’s Roger Waters says he’s ‘far, far,...  17-7-2022   
1      Music  I analyzed the top 100 most popular songs on r...  17-7-2022   
2      Music  Who is a musician or band that you don’t under...  17-7-2022   
3      Music  If I wanted to know what Alabama Shakes is all...  17-7-2022   
4      Music               Music recs for people tripping balls  17-7-2022   
5      Music  My friend makes edm music. He's been going thr...  17-7-2022   
6      Music  John Lee Hooker - One Bourbon, One Scotch, One...  17-7-2022   
7      Music  The Menzingers - America (You're Freaking Me O...  17-7-2022   
8      Music  Have you ev

In [614]:
app = Flask(__name__)

In [615]:
@app.route('/')
def home_page():
    return render_template('index.html')
@app.route('/television/')
def home_television():
    return dframe


In [616]:
if __name__ == '__main__':
    app.run(port=5055)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5055/ (Press CTRL+C to quit)
[2022-07-17 08:22:17,533] ERROR in app: Exception on /television/ [GET]
Traceback (most recent call last):
  File "c:\Users\benkimz\AppData\Local\Programs\Python\Python39\lib\site-packages\flask\app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\Users\benkimz\AppData\Local\Programs\Python\Python39\lib\site-packages\flask\app.py", line 1519, in full_dispatch_request
    return self.finalize_request(rv)
  File "c:\Users\benkimz\AppData\Local\Programs\Python\Python39\lib\site-packages\flask\app.py", line 1538, in finalize_request
    response = self.make_response(rv)
  File "c:\Users\benkimz\AppData\Local\Programs\Python\Python39\lib\site-packages\flask\app.py", line 1730, in make_response
    raise TypeError(
TypeError: The view function did not return a valid response. The return type must be a string, dict, tuple, Response instance, or WSGI callable, but it was a DataFrame.
127.0.0.1 - - [17